In [134]:
import requests as req
url = "https://api.bithumb.com/v1/candles/minutes/1?market=KRW-BTC&count=1"
d_url = "https://api.bithumb.com/v1/candles/days/1?market=KRW-BTC&count=1"
re = req.get(url).json()[0]
d_re = req.get(d_url).json()[0]

max_p = float(re['high_price'])
min_p = float(re['low_price'])
st_p = float(re['opening_price'])


print()


[{'market': 'KRW-BTC', 'candle_date_time_utc': '2026-02-06T01:25:00', 'candle_date_time_kst': '2026-02-06T10:25:00', 'opening_price': 95377000, 'high_price': 95379000, 'low_price': 95377000, 'trade_price': 95377000, 'timestamp': 1770341105000, 'candle_acc_trade_price': 10305253.71069, 'candle_acc_trade_volume': 0.10804757, 'unit': 1}]


103,732,051원


In [153]:
Me = 'java'
comA = ['Java' , 'HTML' , 'javascript','ORACLE']
comB = ['Html' , 'Python' , 'C','Mysql']
comC = ['Java' , 'CSS' , 'javascript', 'C++','ORACLE' , 'PYTHON']




# print(upper(comA[0]))
for i in comA:
    comA = []
    comA.append(i)


# print("합격" if Me in comA else "불합격", '니똥굵다')
# print("합격" if Me in comB else "불합격", '니똥굵다')
# print("합격" if Me in comC else "불합격", '니똥굵다')

java


In [11]:
import requests as req
import webbrowser as wb

# 위성위치
url = "http://api.open-notify.org/iss-now.json"
# 탑승자 명단
url2 = "http://api.open-notify.org/astros.json"

result = req.get(url).json()

print(result)
lat = result['iss_position']['latitude'] # 위도
lon = result['iss_position']['longitude'] # 경도

human_num = req.get(url2).json()["number"]

url = f"https://www.google.com/maps?q={lat},{lon}"
print(f"현재 {human_num}명이 탑승한 인공위성(국제우주정거장) 위치를 열겠습니다.위도:{lat}, 경도:{lon}")
wb.open(url)

{'timestamp': 1770354497, 'iss_position': {'latitude': '8.8014', 'longitude': '21.5709'}, 'message': 'success'}
현재 12명이 탑승한 인공위성(국제우주정거장) 위치를 열겠습니다.위도:8.8014, 경도:21.5709


True

gio: https://www.google.com/maps?q=8.8014,21.5709: Operation not supported


In [13]:
import requests as req

url = "https://api.open-meteo.com/v1/forecast?latitude=37.5665&longitude=126.9780&current_weather=true"

rs = req.get(url).json()
print(rs)

{'latitude': 37.55, 'longitude': 127.0, 'generationtime_ms': 0.06532669067382812, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 34.0, 'current_weather_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature': '°C', 'windspeed': 'km/h', 'winddirection': '°', 'is_day': '', 'weathercode': 'wmo code'}, 'current_weather': {'time': '2026-02-06T05:15', 'interval': 900, 'temperature': -4.9, 'windspeed': 9.6, 'winddirection': 304, 'is_day': 1, 'weathercode': 0}}


In [15]:
import re
import requests 
from bs4 import BeautifulSoup

URL = "https://www.pknu.ac.kr/main/399?action=view&no=723270"

def fetch_text(url: str) -> str:
    # 사이트가 가끔 UA 없으면 막는 경우가 있어 넣어줌
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                      "(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    r = requests.get(url, headers=headers, timeout=10)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")
    # 본문 전체를 텍스트로 (줄바꿈 유지)
    return soup.get_text("\n")

def extract_section(text: str, start_title: str, end_title: str) -> str:
    start = text.find(start_title)
    if start == -1:
        raise ValueError(f"start_title not found: {start_title}")
    end = text.find(end_title, start)
    if end == -1:
        # 끝 타이틀 못 찾으면 페이지 끝까지
        end = len(text)
    return text[start:end]

def parse_daily_menu(section_text: str) -> dict:
    # 예: "2월 2일" 패턴
    date_pat = re.compile(r"(\d+월\s*\d+일)")
    lines = [ln.strip() for ln in section_text.splitlines() if ln.strip()]

    # 날짜가 나오는 라인 인덱스 수집
    date_idxs = []
    for i, ln in enumerate(lines):
        if date_pat.fullmatch(ln):
            date_idxs.append(i)

    if not date_idxs:
        raise ValueError("No date lines like '2월 2일' found in section")

    result = {}
    for idx_pos, i in enumerate(date_idxs):
        date = lines[i]
        j = date_idxs[idx_pos + 1] if idx_pos + 1 < len(date_idxs) else len(lines)

        # 날짜 다음 줄부터 다음 날짜 전까지 = 그날 메뉴 텍스트
        block = lines[i + 1 : j]

        # 너무 일반적인 안내/전화번호 같은 건 필요하면 필터링
        # (원하면 필터 규칙 더 촘촘히 잡아줄게)
        result[date] = block

    return result

if __name__ == "__main__":
    text = fetch_text(URL)

    # "미래관 라일락 레스토랑" 구간만 자르기
    section = extract_section(
        text,
        start_title="미래관 라일락 레스토랑",
        end_title="다래락 학생식당"
    )

    menu_by_day = parse_daily_menu(section)

    # 예: 2월 2일 메뉴만 출력
    day = "2월 2일"
    print(day)
    for item in menu_by_day.get(day, []):
        print("-", item)

ValueError: start_title not found: 미래관 라일락 레스토랑

In [79]:
import requests as rq
url = "https://api.fda.gov/drug/event.json?count=patient.reaction.reactionmeddrapt.exact"

rs = rq.get(url).json()



# aa = input("찾고싶은 질병의 이름을 입력하시오")
# bb = input("찾고싶은 질병의 이름을 입력하시오")
# cc = []

for item in rs['results']:
    if item.get("term") == aa :
        print('보고건의 수는 ' , item.get('count'),'개 입니다.')
        cc.append(item.get('count'))

for item in rs['results']:
    if item.get("term") == bb :
        print('보고건의 수는 ' , item.get('count'),'개 입니다.')
        cc.append(item.get('count'))

if int(cc[0]) - int(cc[1]) > 0:
   c_n = cc[0]
else:
   c_n = cc[1]
    

if cc[0] - cc[1] < 0:
    c = cc[1] - cc[0]
else:
    c = cc[0] - cc[1]
    
print(f"더 많이 발생된 질병은 {c_n} 이고 차이는 {c}건 입니다. ")



pda = rs['meta']['disclaimer'].count("PDA")


print(f"경고문구 안에 PDA의 개수는 {pda}개 입니다.")




보고건의 수는  171703 개 입니다.
보고건의 수는  171703 개 입니다.
더 많이 발생된 질병은 171703 이고 차이는 0건 입니다. 
경고문구 안에 PDA의 개수는 0개 입니다.
